# 소상공인시장진흥공단_상가(상권)정보_API
# https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15012005

## 업종코드
## https://www.data.go.kr/data/15067631/fileData.do

In [ ]:
import requests
import json
import time
import pickle
import pandas as pd

In [ ]:
s_key = "46kyRJ49WNDCbot3XpPvourHjckhU7JeA1KnF66ZyI0ptMlZNGMacC/o7Am+Pv3Y5x5KjEChjhC1D0yOboJ3Sg=="
page = 1

In [ ]:
# 소분류로 검색: storeListInUpjong
# 소분류 코드: I21006,후라이드/양념치킨
# 마지막 수집일: 2023.08.21
# chickenShopsList[0]['body']['totalCount']

In [ ]:
chickenShopLists = []

while True:

    url = 'http://apis.data.go.kr/B553077/api/open/sdsc2/storeListInUpjong'
    params ={'serviceKey' : s_key, 'pageNo' : page, 'numOfRows' : '1000', 'divId' : 'indsSclsCd', 'key' : 'I21006', 'type' : 'json' }

    try:
        r = requests.get(url, params=params)
        print(r.status_code)
        response = r.json()
    except Exception as e:
        print(e)
    finally:
        chickenShopLists.append(response)
        totalcount = chickenShopLists[0]['body']['totalCount']
        total_pages = int(totalcount) // 1000 +1
        print(page, total_pages, end=" ")
        page +=1
        time.sleep(5)
        if page > total_pages:
            break



200
6 41 ("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))
7 41 200
8 41 200
9 41 ("Connection broken: ConnectionResetError(104, 'Connection reset by peer')", ConnectionResetError(104, 'Connection reset by peer'))
10 41 200
11 41 200
12 41 200
13 41 14 41 

KeyboardInterrupt: ignored

In [ ]:
chickenShopLists

[{'header': {'resultCode': '10',
   'resultMsg': 'INVALID_REQUEST_PARAMETER_ERROR'},
  'body': {}}]

In [ ]:
response["body"]['totalCount']

In [ ]:
len(chickenShopLists)

In [ ]:
chickenShopLists[0]

In [ ]:
with open ("chicken_shops.pickle", "wb") as f:
    pickle.dump(chickenShopLists, f)

In [ ]:
with open ("chicken_shops.pickle", "rb") as f:
    data = pickle.load(f)

In [ ]:
data[0]

# for문을 이용한 JSON 자료 언패킹

In [ ]:
#['header']['stdrYm] #자료 기준 년월
# chickenShopsList

Chicken_Shops = []

for p_num, shoplists in enumerate(data):
    for shop in shoplists['body']['items']:
        p_num += 1
        stdrYm = shoplists['header'].get('stdrYm', 'NaN')
        shopId = shop['bizesId']   #가게 ID
        shopname = shop['bizesNm']   #가게명
        shopIdentity = shop['ksicNm']  #가게 세부업종
        shop_address1 = shop['ctprvnNm'] #시
        city_code = shop['ctprvnCd'] # citycode
        shop_address2 = shop['signguNm'] #구
        gu_code = shop['signguCd'] # 구 Code
        shop_address3 = shop['ldongNm'] #법정동
        dong_law_code = shop['ldongCd'] # 법정동 코드
        shop_old_address = shop['lnoAdr'] # 구 주소
        shop_road_address = shop['rdnmAdr']  # 도로명 주소
        shop_floor = shop['flrNo'] #층 정보
        shop_sNo = shop['hoNo']  #호 수
        coordinateY = str(shop['lon'])  #경도
        coordinateX = str(shop['lat'])  # 위도
        shopDatas = p_num, stdrYm, shopId, shopname, shopIdentity, shop_address1, city_code, shop_address2, gu_code, shop_address3, \
                    shop_address3, dong_law_code, shop_old_address, shop_floor, shop_sNo, shop_road_address,coordinateX, coordinateY
        Chicken_Shops.append(shopDatas)
Chicken_Shops

In [ ]:
len(Chicken_Shops)

# JSON 자료를 데이터 프레임으로 저장하기

In [ ]:
#['header']['stdrYm] #자료 기준 년월
# chickenShopsList
dataforMap = []
dataforDB =[]
Chicken_Shops = pd.DataFrame()

for p_num, shoplists in enumerate(data):
    for shop in shoplists['body']['items']:
        p_num += 1
        stdrYm = shoplists['header'].get('stdrYm', 'NaN')
        Chicken_Shop = pd.DataFrame(shop, index=[p_num])
        Chicken_Shop['stdrYm'] = stdrYm
        Chicken_Shops = pd.concat([Chicken_Shops, Chicken_Shop])

Chicken_Shops

In [ ]:
Chicken_Shops.to_csv("Chicken_shop_list20111121.csv", index = False, encoding="utf-8" )

In [ ]:
Chicken_Shops

In [ ]:
Chicken_Shops.columns

In [ ]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

In [ ]:
import sqlite3

In [ ]:
con = sqlite3.connect("./datas/sqlite3_Chicken_shops.db")
c = con.cursor()

In [ ]:
c.execute('CREATE TABLE IF NOT EXISTS Chicken_shops (bizesId, bizesNm, brchNm, indsLclsCd, indsLclsNm, indsMclsCd, indsMclsNm, indsSclsCd, indsSclsNm, ksicCd, ksicNm, ctprvnCd, ctprvnNm, signguCd, signguNm, adongCd, adongNm, ldongCd, ldongNm, lnoCd, plotSctCd, plotSctNm, lnoMnno, lnoSlno, lnoAdr, rdnmCd, rdnm, bldMnno, bldSlno, bldMngNo, bldNm, rdnmAdr, oldZipcd, newZipcd, dongNo, flrNo, hoNo, lon, lat, stdrYm)')
con.commit()

In [ ]:
Chicken_Shops.to_sql('Chicken_shops', con, if_exists='replace', index = False)

In [ ]:
c.execute('''
SELECT * FROM Chicken_shops
          ''')

for row in c.fetchall():
    print (row)

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///datas/Chicken_shops2.db', echo=False)
conn = engine.raw_connection()
Chicken_Shops.to_sql('Chicken_Shops2', con=conn, if_exists='append')

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///datas/Chicken_shops2.db', echo=False)
conn = engine.raw_connection()

In [ ]:
test = pd.read_sql('select * from Chicken_Shops2', conn)
test

In [ ]:
test

In [ ]:
test.groupby('ctprvnNm')['bizesNm'].count().sort_values(ascending=False)

In [ ]:
print(test.groupby(['ctprvnNm','signguNm'])['bizesNm'].count().sort_values(ascending=False))

In [ ]:
test.groupby(['bizesNm','ctprvnNm'])['ctprvnNm'].count().sort_values(ascending=False)

In [ ]:
test['bizesNm','ctprvnNm'].value_counts()

In [ ]:
test.sort_values()